In [ ]:
# main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import numpy as np
import pandas as pd
from typing import List

# Загружаем сохраненные модели и трансформеры
model = joblib.load('model.pkl')
scaler = joblib.load('scaler.pkl')
encoder = joblib.load('encoder.pkl')

app = FastAPI(title="ML Prediction API")

# Модель для входных данных
class InputData(BaseModel):
    age: float
    income: float
    category: str
    feature1: float
    feature2: float

# Модель для ответа
class Prediction(BaseModel):
    prediction: float
    probability: float

@app.get("/")
def read_root():
    return {"message": "ML Prediction API is running"}

@app.post("/predict", response_model=Prediction)
def predict(data: InputData):
    try:
        # Преобразуем входные данные
        df = pd.DataFrame([data.dict()])
        
        # Применяем трансформации
        df['category_encoded'] = encoder.transform(df[['category']])
        df = df.drop('category', axis=1)
        
        numerical_features = ['age', 'income', 'feature1', 'feature2', 'category_encoded']
        df[numerical_features] = scaler.transform(df[numerical_features])
        
        # Прогноз
        prediction = model.predict(df)[0]
        probability = model.predict_proba(df)[0][1] if hasattr(model, 'predict_proba') else 0.5
        
        return {
            "prediction": float(prediction),
            "probability": float(probability)
        }
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.get("/health")
def health_check():
    return {"status": "healthy"}

pip install fastapi uvicorn
uvicorn main:app --reload

http://localhost:8000/docs

# Сохранение моделей

In [ ]:
# train_and_save.py
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Обучаем модели
model = RandomForestClassifier()
model.fit(X_train, y_train)

scaler = StandardScaler()
scaler.fit(X_train)

encoder = OneHotEncoder()
encoder.fit(categorical_data)

# Сохраняем всё
joblib.dump(model, 'model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(encoder, 'encoder.pkl')

print("Модели сохранены!")